## Documentation

To read more about analyzers, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-analyze.html).

![analyzer_api](../images/analyzer_api.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(hosts=HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'iugjHCt8SwCWRVd35xnJ0A',
 'name': '5013781c82bc',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-02-05T22:10:57.067596412Z',
             'build_flavor': 'default',
             'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.2'}}


## 1. Character filters

### 1.1. HTML Strip Character Filter

In [2]:
from pprint import pprint

response = es.indices.analyze(
    char_filter=["html_strip"],
    text="I&apos;m so happy</b>!</p>"
)

pprint(response.body)

{'tokens': [{'end_offset': 26,
             'position': 0,
             'start_offset': 0,
             'token': "I'm so happy!\n",
             'type': 'word'}]}


### 1.2. Mapping character filter

In [3]:
response = es.indices.analyze(
    tokenizer="keyword",
    char_filter=[
        {
            "type": "mapping",
            "mappings": [
                "٠ => 0",
                "١ => 1",
                "٢ => 2",
                "٣ => 3",
                "٤ => 4",
                "٥ => 5",
                "٦ => 6",
                "٧ => 7",
                "٨ => 8",
                "٩ => 9"
            ]
        }
    ],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)

pprint(response.body)

{'tokens': [{'end_offset': 37,
             'position': 0,
             'start_offset': 0,
             'token': 'I saw comet Tsuchinshan Atlas in 2024',
             'type': 'word'}]}


## 2. Tokenizer

Read more about tokenizers [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenizers.html).

### 2.1. Standard

In [4]:
response = es.indices.analyze(
    tokenizer="standard",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'The', Type: <ALPHANUM>
Token '2', Type: <NUM>
Token 'QUICK', Type: <ALPHANUM>
Token 'Brown', Type: <ALPHANUM>
Token 'Foxes', Type: <ALPHANUM>
Token 'jumped', Type: <ALPHANUM>
Token 'over', Type: <ALPHANUM>
Token 'the', Type: <ALPHANUM>
Token 'lazy', Type: <ALPHANUM>
Token 'dog's', Type: <ALPHANUM>
Token 'bone', Type: <ALPHANUM>


### 2.2. Lowercase

In [5]:
response = es.indices.analyze(
    tokenizer="lowercase",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'the', Type: word
Token 'quick', Type: word
Token 'brown', Type: word
Token 'foxes', Type: word
Token 'jumped', Type: word
Token 'over', Type: word
Token 'the', Type: word
Token 'lazy', Type: word
Token 'dog', Type: word
Token 's', Type: word
Token 'bone', Type: word


### 2.3. Whitespace

In [7]:
response = es.indices.analyze(
    tokenizer="whitespace",
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'The', Type: word
Token '2', Type: word
Token 'QUICK', Type: word
Token 'Brown-Foxes', Type: word
Token 'jumped', Type: word
Token 'over', Type: word
Token 'the', Type: word
Token 'lazy', Type: word
Token 'dog's', Type: word
Token 'bone.', Type: word


## 3. Token filter

Read more about token filters [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-tokenfilters.html).

### 3.1. Apostrophe

In [9]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=["apostrophe"],
    text="The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'The', Type: <ALPHANUM>
Token '2', Type: <NUM>
Token 'QUICK', Type: <ALPHANUM>
Token 'Brown', Type: <ALPHANUM>
Token 'Foxes', Type: <ALPHANUM>
Token 'jumped', Type: <ALPHANUM>
Token 'over', Type: <ALPHANUM>
Token 'the', Type: <ALPHANUM>
Token 'lazy', Type: <ALPHANUM>
Token 'dog', Type: <ALPHANUM>
Token 'bone', Type: <ALPHANUM>


### 3.2. Decimal digit

In [10]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=["decimal_digit"],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'I', Type: <ALPHANUM>
Token 'saw', Type: <ALPHANUM>
Token 'comet', Type: <ALPHANUM>
Token 'Tsuchinshan', Type: <ALPHANUM>
Token 'Atlas', Type: <ALPHANUM>
Token 'in', Type: <ALPHANUM>
Token '2024', Type: <NUM>


### 3.3. Reverse

In [11]:
response = es.indices.analyze(
    tokenizer="standard",
    filter=["reverse"],
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'I', Type: <ALPHANUM>
Token 'was', Type: <ALPHANUM>
Token 'temoc', Type: <ALPHANUM>
Token 'nahsnihcusT', Type: <ALPHANUM>
Token 'saltA', Type: <ALPHANUM>
Token 'ni', Type: <ALPHANUM>
Token '٤٢٠٢', Type: <NUM>


## 4. Built-in analyzers

Read more about token filters [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-analyzers.html).

### 4.1. Standard

In [15]:
response = es.indices.analyze(
    analyzer="standard",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'i', Type: <ALPHANUM>
Token 'saw', Type: <ALPHANUM>
Token 'comet', Type: <ALPHANUM>
Token 'tsuchinshan', Type: <ALPHANUM>
Token 'atlas', Type: <ALPHANUM>
Token 'in', Type: <ALPHANUM>
Token '٢٠٢٤', Type: <NUM>


### 4.2. Stop

In [16]:
response = es.indices.analyze(
    analyzer="stop",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}', Type: {token['type']}")

Token 'i', Type: word
Token 'saw', Type: word
Token 'comet', Type: word
Token 'tsuchinshan', Type: word
Token 'atlas', Type: word


### 4.3. Keyword

In [17]:
response = es.indices.analyze(
    analyzer="keyword",
    text="I saw comet Tsuchinshan Atlas in ٢٠٢٤"
)
tokens = response.body["tokens"]
for token in tokens:
    print(f"Token '{token['token']}'")

Token 'I saw comet Tsuchinshan Atlas in ٢٠٢٤'


## 5. Index time VS Search time analysis

### 5.1. Index time

Index-time analysis transforms text before it's stored in the index. In this example, let's create an index with an analyzer that lowercases text, removes HTML tags, and replaces ampersands (&) with the word "and."

In [21]:
index_name = "index_time_example"
body = {
    "settings": {
        "analysis": {
            "char_filter": {
                "ampersand_replacement": {
                    "type": "mapping",
                    "mappings": ["& => and"]
                }
            },
            "analyzer": {
                "custom_index_analyzer": {
                    "type": "custom",
                    "char_filter": ["html_strip", "ampersand_replacement"],
                    "tokenizer": "standard",
                    "filter": ["lowercase"]
                }
            }
        },
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "custom_index_analyzer"
            }
        }
    },
}


es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=body)

document = {
    "content": "Visit my website https://kosamtech.com/ & like some images!"}
response = es.index(index=index_name, id=1, body=document)
pprint(response.body)

{'_id': '1',
 '_index': 'index_time_example',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_version': 1,
 'result': 'created'}


When searching for the document, you'll notice that the content appears unchanged. This is expected because Elasticsearch stores the transformed tokens in an inverted index for searching purposes, while keeping the original document intact in the `_source` field.

In [22]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    }
)
hits = response["hits"]["hits"]

for hit in hits:
    print(hit["_source"])

{'content': 'Visit my website https://kosamtech.com/ & like some images!'}


We can verify that the custom analyzer is working by applying it to the document like this.

In [24]:
response = es.indices.analyze(
    index=index_name,
    body={
        "field": "content",
        "text": "Visit my website https://kosamtech.com/ & like some images!"
    }
)

tokens = response["tokens"]
for token in tokens:
    print(f"Token: '{token['token']}")

Token: 'visit
Token: 'my
Token: 'website
Token: 'https
Token: 'kosamtech.com
Token: 'and
Token: 'like
Token: 'some
Token: 'images


### 5.2. Search time

Search-time analysis transforms text only when a search query is performed, not when data is indexed. In this example, we’ll perform a search with a search-time analyzer that transforms text differently (e.g., it lowercases and removes stop words).

In [25]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "match": { # match is used for full-text search
                "content": {
                    "query": "kosamtech.com",
                    "analyzer": "standard" # Using a different analyzer then one used at index time
                }
            }
        }
    }
)

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])


{'content': 'Visit my website https://kosamtech.com/ & like some images!'}


You can also use a `term` query to match exact terms. Since `kosamtech.com` exists exactly as-is in the document, this query will return the document in the results.

In [26]:
response = es.search(index=index_name, body={
    "query": {
        "term": { # term is used for exact matches
            "content": {
                "value": "kosamtech.com",
            }
        }
    }
})

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])

{'content': 'Visit my website https://kosamtech.com/ & like some images!'}


In this case, `KOSAMTECH.com` does not appear in the document, so no results are returned.

In [27]:
response = es.search(index=index_name, body={
    "query": {
        "term": { # term is used for exact matches
            "content": {
                "value": "KOSAMTECH.com",
            }
        }
    }
})

hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_source"])